In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd #Basic library for all of our dataset operations
import numpy as np
import requests
import io
import statsmodels.tsa.api as smt
import statsmodels as sm
import tensorflow as tf
import pmdarima as pm
import warnings
import xgboost as xgb

import lightgbm as lgb
import gluonts
from math import sqrt
import warnings
warnings.filterwarnings('ignore')
# # import shap
# # warnings.filterwarnings("ignore") #We will use deprecated models of statmodels which throw a lot of warnings to use more modern ones

from utils.metrics import evaluate
from utils.plots import bar_metrics

from statsmodels.tsa.ar_model import AR
from random import random
from datetime import datetime
from fbprophet import Prophet
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing
from sklearn import linear_model, svm
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, make_scorer
from sklearn.model_selection import cross_val_score, GridSearchCV
from math import sqrt
from xgboost import plot_importance, plot_tree
from statsmodels.tsa.arima_model import ARIMA
from gluonts.model.deepar import DeepAREstimator
from gluonts.trainer import Trainer
from gluonts.dataset.common import ListDataset
from gluonts.evaluation.backtest import make_evaluation_predictions
from itertools import islice
from pylab import rcParams
# progress bar
from tqdm import tqdm, tqdm_notebook
# from bayes_opt import BayesianOptimization


#Extra settings
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)
plt.style.use('bmh')
mpl.rcParams['axes.labelsize'] = 14
mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['ytick.labelsize'] = 12
mpl.rcParams['text.color'] = 'k'


from datetime import datetime

import matplotlib.pyplot as plt
import seaborn

from sklearn.metrics import mean_squared_error
from math import sqrt
from statsmodels.tsa.stattools import adfuller,kpss
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA

from pmdarima.arima import auto_arima
import statsmodels.graphics.tsaplots as tsaplot
from statsmodels.tsa.holtwinters import Holt, ExponentialSmoothing, SimpleExpSmoothing
from statsmodels.graphics.tsaplots import plot_pacf
from itertools import permutations
from fbprophet import Prophet

from sklearn.linear_model import TweedieRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import ElasticNet

import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import itertools
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
from pylab import rcParams

rcParams['figure.figsize'] = 18, 8
warnings.filterwarnings("ignore")

from statsmodels.tsa.seasonal import seasonal_decompose

#-----------------------Imports from Sktime-------------------------
from sktime.forecasting.base import ForecastingHorizon
from sktime.transformations.series.detrend import Deseasonalizer, Detrender
from sktime.forecasting.trend import PolynomialTrendForecaster
from sktime.forecasting.model_selection import (
    temporal_train_test_split,
)
from sktime.utils.plotting import plot_series
from sktime.forecasting.compose import (
    TransformedTargetForecaster,
    ReducedRegressionForecaster
)

Importing plotly failed. Interactive plots will not work.


In [2]:

rfg = RandomForestRegressor( random_state=0)


BR = linear_model.BayesianRidge()


Lasso = linear_model.Lasso(alpha=0.1)


XGB = xgb.XGBRegressor(objective ='reg:squarederror',n_estimators=1000)


lgbm = lgb.LGBMRegressor()


SVR = svm.SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)


KNN = KNeighborsRegressor(n_neighbors=2)


Tweedie = TweedieRegressor()

SGD = SGDRegressor()

GBDT = GradientBoostingRegressor()

ExtraTree = ExtraTreesRegressor()

EN = ElasticNet()

rgs = [rfg, Lasso, XGB, KNN, ExtraTree]
names = ['rfg', 'Lasso', 'XGB', 'KNN', 'ExtraTree']

In [3]:
jetoil = pd.read_csv('jetoil.csv')

jetoil = jetoil[:-1]
jetoil['price'] = jetoil['price'].values.astype('float')


jetoil_date = jetoil.copy().date
for itr in range(jetoil.shape[0]):
    jetoil_date[itr] =  np.datetime64(datetime.strptime(jetoil.date[itr], "%Y/%m/%d").date())
    
jetoil.date = jetoil_date
jetoil['date'] = pd.to_datetime(jetoil['date'], errors='coerce')
jetoil = jetoil.set_index('date')

trend = pd.read_csv('trend.csv')

trend = trend[:-1]

trend['search'] = trend['search'].values.astype('int')


trend_date = trend.copy().date
for itr in range(trend.shape[0]):
    trend_date[itr] =  np.datetime64(datetime.strptime(trend.date[itr], "%Y/%m/%d").date())
    
trend.date = trend_date
trend['date'] = pd.to_datetime(trend['date'], errors='coerce')
trend = trend.set_index('date')

im = pd.read_csv('import.csv')

im['import'] = im['import'].values.astype('int')

im_date = im.copy().date
for itr in range(im.shape[0]):
    im_date[itr] =  np.datetime64(datetime.strptime(im.date[itr], "%Y/%m/%d").date())
    
im.date = im_date
im['date'] = pd.to_datetime(im['date'], errors='coerce')
im = im.set_index('date')



In [4]:
raw_col = ['date', 'highest', 'high', 'mid', 'low', 'ave', 'ave_dif', 'volume', 'vol_dif', 'resid']
raw = pd.read_csv('FS443.csv',names=raw_col).drop(['ave_dif', 'vol_dif', 'resid'], axis=1)
for i in range(raw.shape[0]):
    raw.volume[i] = int(raw.volume[i].split(' ')[0].replace(',', ''))
raw.volume = raw.volume.values.astype('int')

ts_date = raw.copy().date
for itr in range(raw.shape[0]):
    date = raw.date[itr].split('/')
    date[0] = str(int(date[0]) + 1911)
    result = '/'.join(date)
    ts_date[itr] =  np.datetime64(datetime.strptime(result, "%Y/%m/%d").date())
    


ts = raw.copy()
ts.date = ts_date
ts['date'] = pd.to_datetime(ts['date'], errors='coerce')
ts = ts.set_index('date')


ts = ts.drop(ts[ts.ave == 0].index)



In [21]:
im

,import
date,
2012-01-01,2558725
2012-02-01,3778725
2012-03-01,4084958
2012-04-01,3611260
2012-05-01,4244055
...,...
2020-08-01,3913025
2020-09-01,4684160
2020-10-01,8486150


In [59]:
from datetime import datetime, timedelta
import dateutil.relativedelta


In [30]:
cut = pd.Timestamp(np.datetime64(datetime.strptime('2012/03/01', "%Y/%m/%d").date()))


In [32]:
cut_ts = ts[ts.index > cut]

In [90]:
im_num = []
oil_p = []
search = []
for date in cut_ts.index:
    sub2 = date.to_pydatetime()- dateutil.relativedelta.relativedelta(months=2)
    y = sub2.year
    m = sub2.month
    sub1 = date.to_pydatetime()- dateutil.relativedelta.relativedelta(months=1)
    y1 = sub1.year
    m1 = sub1.month
    im_num.append(im[(im.index.year == y) & (im.index.month == m)].values.ravel()[0])
    oil_p.append(jetoil[(jetoil.index.year == y) & (jetoil.index.month == m)].values.ravel()[0])
    search.append(trend[(trend.index.year == y1) & (trend.index.month == m1)].values.ravel()[0])

In [91]:
cut_ts['im'] = im_num
cut_ts['oil'] = oil_p
cut_ts['search'] = search

In [92]:
cut_ts

,highest,high,mid,low,ave,volume,im,oil,search
date,,,,,,,,,
2012-03-02,255,216,185,149,184.0,747,2558725,3.09,0
2012-03-03,240,230,174,121,174.0,986,2558725,3.09,0
2012-03-05,251,222,166,113,166.0,964,2558725,3.09,0
2012-03-06,211,203,159,113,158.0,605,2558725,3.09,0
2012-03-07,214,195,141,116,146.0,438,2558725,3.09,0
...,...,...,...,...,...,...,...,...,...
2021-01-15,305,305,305,305,305.0,48,4239795,1.13,38
2021-01-16,330,330,320,239,305.4,97,4239795,1.13,38
2021-01-18,327,327,327,327,327.0,96,4239795,1.13,38


In [94]:
cut_ts.index = pd.RangeIndex(len(cut_ts))

In [5]:
raw_col = ['date', 'highest', 'high', 'mid', 'low', 'ave', 'ave_dif', 'volume', 'vol_dif', 'resid']
raw = pd.read_csv('FS443.csv',names=raw_col).drop(['ave_dif', 'vol_dif', 'resid'], axis=1)
for i in range(raw.shape[0]):
    raw.volume[i] = int(raw.volume[i].split(' ')[0].replace(',', ''))
raw.volume = raw.volume.values.astype('int')

ts_date = raw.copy().date
for itr in range(raw.shape[0]):
    date = raw.date[itr].split('/')
    date[0] = str(int(date[0]) + 1911)
    result = '/'.join(date)
    ts_date[itr] =  np.datetime64(datetime.strptime(result, "%Y/%m/%d").date())
    


ts = raw.copy()
ts.date = ts_date
ts['date'] = pd.to_datetime(ts['date'], errors='coerce')
ts = ts.set_index('date')
ts = ts.drop(ts[ts.ave == 0].index) ## bugg
ts.index = pd.RangeIndex(len(ts))






In [6]:
ts

,highest,high,mid,low,ave,volume
0,195,131,77,59,84.0,327
1,165,144,111,93,113.0,456
2,204,160,82,65,94.0,432
3,189,161,114,80,116.0,583
4,181,157,90,55,96.0,1052
...,...,...,...,...,...,...
6670,305,305,305,305,305.0,48
6671,330,330,320,239,305.4,97
6672,327,327,327,327,327.0,96
6673,212,211,201,201,203.3,69


In [5]:
train = ts[:-28]
test = ts[-28:-14]

In [39]:

a = ts[:-14].ave.values

In [43]:
a.mean()

141.07129560126106

In [7]:
train_list = []
test_list = []
for i in range(1,6,1):
    train_list.append(ts[:-14*i])
    if i != 1:
        test_list.append(ts[-14*i:-14*i+14])
    else:
        test_list.append(ts[-14*i:])

In [8]:

for i in range(len(rgs)):

    for window in range(40,51):
        rmse_list = []
        print('window:',window)
        for j in range(len(train_list)):
        
            regressor = rgs[i]
            name = names[i]
            forecaster = ReducedRegressionForecaster(
            regressor=regressor, window_length=window, strategy="recursive"
            )
            fh = ForecastingHorizon(test_list[j].index, is_relative=False)
            forecaster.fit(train_list[j].ave)
            y_pred = forecaster.predict(fh)
            mse = ((test_list[j].ave - y_pred) ** 2).mean()
            rmse_list.append(round(np.sqrt(mse), 2))
            
        print(names[i], 'RMSE: {}'.format(np.array(rmse_list).mean()))
    print('\n')

window: 40
rfg RMSE: 45.482000000000006
window: 41
rfg RMSE: 44.495999999999995
window: 42
rfg RMSE: 39.83
window: 43
rfg RMSE: 41.802
window: 44
rfg RMSE: 42.604
window: 45
rfg RMSE: 47.019999999999996
window: 46
rfg RMSE: 45.843999999999994
window: 47
rfg RMSE: 39.204
window: 48
rfg RMSE: 40.944
window: 49
rfg RMSE: 43.95399999999999
window: 50
rfg RMSE: 40.745999999999995


window: 40
Lasso RMSE: 42.08200000000001
window: 41
Lasso RMSE: 41.928000000000004
window: 42
Lasso RMSE: 42.122
window: 43
Lasso RMSE: 42.052
window: 44
Lasso RMSE: 42.102
window: 45
Lasso RMSE: 42.227999999999994
window: 46
Lasso RMSE: 42.21
window: 47
Lasso RMSE: 42.376
window: 48
Lasso RMSE: 42.362
window: 49
Lasso RMSE: 42.358
window: 50
Lasso RMSE: 42.326


window: 40
XGB RMSE: 53.476
window: 41
XGB RMSE: 51.222
window: 42
XGB RMSE: 46.658
window: 43
XGB RMSE: 53.74799999999999
window: 44
XGB RMSE: 54.041999999999994
window: 45
XGB RMSE: 48.864
window: 46
XGB RMSE: 48.706
window: 47
XGB RMSE: 47.098
window:

In [81]:

for i in range(len(rgs)):

    for window in range(3,15):
        rmse_list = []
        print('window:',window)
        for j in range(len(train_list)):
        
            regressor = rgs[i]
            name = names[i]
            forecaster = ReducedRegressionForecaster(
            regressor=regressor, window_length=window, strategy="recursive"
            )
            fh = ForecastingHorizon(test_list[j].index, is_relative=False)
            forecaster.fit(train_list[j].ave)
            y_pred = forecaster.predict(fh)
            mse = ((test_list[j].ave - y_pred) ** 2).mean()
            rmse_list.append(round(np.sqrt(mse), 2))
            
        print(names[i], 'RMSE: {}'.format(np.array(rmse_list).mean()))
    print('\n')

window: 3
rfg RMSE: 40.546
window: 4
rfg RMSE: 47.2
window: 5
rfg RMSE: 38.05
window: 6
rfg RMSE: 38.20399999999999
window: 7
rfg RMSE: 37.79600000000001
window: 8
rfg RMSE: 34.768
window: 9
rfg RMSE: 36.582
window: 10
rfg RMSE: 34.221999999999994
window: 11
rfg RMSE: 33.980000000000004
window: 12
rfg RMSE: 36.446
window: 13
rfg RMSE: 34.391999999999996
window: 14
rfg RMSE: 45.176


window: 3
Lasso RMSE: 43.11
window: 4
Lasso RMSE: 42.464
window: 5
Lasso RMSE: 42.116
window: 6
Lasso RMSE: 42.025999999999996
window: 7
Lasso RMSE: 42.166000000000004
window: 8
Lasso RMSE: 41.958
window: 9
Lasso RMSE: 41.804
window: 10
Lasso RMSE: 41.784
window: 11
Lasso RMSE: 41.912
window: 12
Lasso RMSE: 41.972
window: 13
Lasso RMSE: 41.84400000000001
window: 14
Lasso RMSE: 41.824


window: 3
XGB RMSE: 41.094
window: 4
XGB RMSE: 38.76800000000001
window: 5
XGB RMSE: 44.624
window: 6
XGB RMSE: 66.97200000000001
window: 7
XGB RMSE: 48.008
window: 8
XGB RMSE: 49.791999999999994
window: 9
XGB RMSE: 44.152
wi

In [46]:
train = train_list[0]
test = test_list[0]

In [52]:
train_list[1]

,highest,high,mid,low,ave,volume
0,195,131,77,59,84.0,327
1,165,144,111,93,113.0,456
2,204,160,82,65,94.0,432
3,189,161,114,80,116.0,583
4,181,157,90,55,96.0,1052
...,...,...,...,...,...,...
6647,193,193,148,131,153.7,41
6649,173,173,173,173,173.0,48
6650,190,185,174,172,175.8,146
6651,153,149,137,132,138.4,160


In [54]:
test_list[1]

,highest,high,mid,low,ave,volume
6653,123,123,120,115,119.9,41
6654,162,160,132,128,136.8,78
6655,175,162,122,100,125.7,71
6656,148,148,142,137,142.0,22
6657,161,161,161,161,161.0,15
6658,154,154,154,154,154.0,13
6659,230,230,203,168,201.4,26
6660,226,226,205,182,204.7,31
6661,233,233,199,150,195.9,60
6662,183,183,161,121,156.8,77


In [69]:
y_pred

6667    253.973
6668    237.491
6669    216.563
6670    221.737
6671    216.151
6672    225.289
6673    229.116
6674    237.071
6675    233.060
6676    224.333
6677    220.554
6678    222.135
6679    212.220
6680    206.226
dtype: float64

In [76]:
train_list[1].ave

0        84.0
1       113.0
2        94.0
3       116.0
4        96.0
        ...  
6647    153.7
6649    173.0
6650    175.8
6651    138.4
6652    136.8
Name: ave, Length: 6647, dtype: float64

In [77]:
test_list[1].ave

6653    119.9
6654    136.8
6655    125.7
6656    142.0
6657    161.0
6658    154.0
6659    201.4
6660    204.7
6661    195.9
6662    156.8
6663    165.4
6664    187.0
6665    183.0
6666    265.0
Name: ave, dtype: float64

In [80]:
i = 0
j = 1
window = 5
regressor = RandomForestRegressor( )
name = names[i]
forecaster = ReducedRegressionForecaster(
regressor=regressor, window_length=window, strategy="recursive"
)
fh = ForecastingHorizon(test_list[j].index, is_relative=False)
forecaster.fit(train_list[j].ave)
y_pred = forecaster.predict(fh)
mse = ((test_list[j].ave - y_pred) ** 2).mean()

print(names[i], 'RMSE: {}'.format(round(np.sqrt(mse), 2)))

rfg RMSE: 50.6


In [65]:
for val in train_list[j].ave.values:
    print(val)

84.0
113.0
94.0
116.0
96.0
113.0
159.0
140.0
166.0
146.0
163.0
160.0
146.0
146.0
165.0
208.0
195.0
184.0
166.0
150.0
175.0
203.0
206.0
192.0
181.0
160.0
179.0
164.0
177.0
181.0
185.0
190.0
221.0
229.0
219.0
191.0
157.0
174.0
189.0
206.0
180.0
198.0
183.0
196.0
192.0
184.0
162.0
145.0
126.0
136.0
142.0
117.0
105.0
101.0
108.0
97.0
104.0
114.0
148.0
188.0
181.0
139.0
161.0
202.0
205.0
261.0
253.0
138.0
125.0
149.0
152.0
158.0
180.0
139.0
144.0
122.0
158.0
171.0
186.0
173.0
160.0
172.0
156.0
149.0
167.0
189.0
169.0
174.0
177.0
189.0
193.0
161.0
145.0
139.0
136.0
154.0
135.0
124.0
96.0
132.0
131.0
173.0
216.0
171.0
149.0
174.0
193.0
180.0
202.0
200.0
182.0
168.0
186.0
136.0
117.0
123.0
121.0
142.0
109.0
114.0
69.0
48.0
46.0
53.0
49.0
49.0
48.0
64.0
58.0
71.0
75.0
97.0
116.0
100.0
88.0
119.0
149.0
147.0
110.0
119.0
91.0
89.0
142.0
110.0
76.0
67.0
85.0
72.0
86.0
99.0
91.0
115.0
114.0
168.0
146.0
153.0
84.0
81.0
139.0
155.0
176.0
173.0
140.0
159.0
147.0
182.0
155.0
131.0
129.0
148.0
144.0
157

218.0
218.0
221.0
199.0
202.0
167.0
154.0
178.0
151.0
151.0
136.0
157.0
166.0
190.0
156.0
150.0
151.0
156.0
148.0
145.0
147.0
145.0
164.0
143.0
168.0
184.0
205.0
204.0
196.0
213.0
203.0
225.0
214.0
187.0
176.0
200.0
137.0
140.0
157.0
160.0
164.0
129.0
107.0
114.0
113.0
142.0
143.0
141.0
136.0
135.0
136.0
112.0
104.0
77.0
100.0
104.0
117.0
107.0
110.0
89.0
74.0
71.0
85.0
73.0
74.0
82.0
104.0
117.0
120.0
97.0
86.0
97.0
96.0
97.0
91.0
80.0
94.0
90.0
125.0
113.0
117.0
114.0
118.0
94.0
99.0
97.0
79.0
82.0
77.0
72.0
77.0
69.0
88.0
124.0
127.0
154.0
139.0
159.0
154.0
80.0
107.0
132.0
123.0
99.0
94.0
108.0
161.0
154.0
161.0
144.0
132.0
133.0
137.0
129.0
151.0
160.0
170.0
211.0
230.0
216.0
200.0
183.0
208.0
203.0
206.0
201.0
176.0
147.0
136.0
134.0
162.0
172.0
145.0
164.0
165.0
158.0
153.0
123.0
107.0
113.0
84.0
86.0
108.0
121.0
140.0
132.0
122.0
101.0
92.0
109.0
106.0
108.0
97.0
83.0
66.0
121.0
121.0
107.0
113.0
105.0
105.0
139.0
150.0
120.0
141.0
129.0
113.0
107.0
106.0
105.0
91.0
83.0
107.0


150.0
200.0
192.0
208.0
209.0
170.0
145.0
119.0
146.0
158.0
144.0
124.0
150.0
150.0
146.0
140.0
139.0
140.0
149.0
184.0
212.0
194.0
167.0
138.0
180.0
182.0
192.0
238.0
235.0
225.0
231.0
243.0
239.0
218.0
183.0
160.0
138.0
145.0
169.0
142.0
150.0
117.0
92.0
78.0
68.0
97.0
89.0
115.0
101.0
93.0
81.0
99.0
110.0
94.0
85.0
86.0
83.0
103.0
90.0
83.0
105.0
102.0
127.0
130.0
129.0
113.0
122.0
124.0
112.0
128.0
128.0
148.0
161.0
191.0
198.0
191.0
179.0
152.0
151.1
167.8
169.9
180.0
142.0
124.0
104.0
119.0
124.0
136.0
106.0
147.0
153.0
151.0
140.0
171.0
197.0
185.0
191.0
169.0
172.0
135.0
114.0
123.0
142.0
137.0
120.0
122.0
103.0
95.0
98.0
85.0
112.0
110.0
110.0
113.0
86.0
90.0
114.0
139.0
83.0
104.0
168.0
123.0
144.0
167.0
131.0
116.0
116.0
122.0
90.0
84.0
84.0
92.0
95.0
97.0
102.0
140.0
109.0
108.0
109.0
117.0
99.0
101.0
124.0
118.0
126.0
84.0
75.0
64.0
71.0
92.0
104.0
101.0
106.0
80.0
92.0
95.0
117.0
108.0
93.0
115.0
136.0
135.0
126.0
115.0
109.0
113.0
123.0
167.0
117.0
114.0
123.0
129.0
154.

134.9
97.6
112.7
86.7
89.9
97.4
85.4
64.7
70.7
68.1
101.1
87.2
71.3
66.6
79.6
79.7
85.5
110.1
143.7
102.7
74.4
85.2
97.9
106.4
99.0
122.6
144.1
138.4
149.4
123.3
146.0
156.5
146.7
124.8
131.4
163.9
180.1
180.6
149.8
148.2
139.0
148.1
139.3
134.2
122.6
114.1
116.5
133.8
118.4
100.9
78.0
44.4
49.2
61.7
81.2
73.2
118.2
135.1
141.1
101.6
95.5
121.2
125.8
129.7
142.5
131.5
111.7
106.4
108.3
97.3
111.7
106.6
113.9
119.2
133.6
156.9
164.8
188.9
189.0
136.9
147.7
140.8
150.7
203.7
250.5
213.8
271.7
258.5
217.7
244.9
239.5
198.9
176.6
181.0
180.8
187.0
199.9
246.4
306.7
269.9
242.9
247.0
262.6
282.2
260.0
241.1
192.1
176.2
197.4
217.2
214.8
231.1
216.7
204.0
191.2
165.6
149.5
165.4
140.8
117.0
130.4
113.6
118.3
114.7
112.1
84.0
153.5
154.4
149.6
168.5
156.2
195.1
189.0
164.0
137.4
124.6
106.2
106.7
98.5
98.4
82.5
87.1
112.4
98.7
94.1
97.6
105.7
117.6
111.8
96.0
110.7
121.0
119.5
134.7
137.3
152.5
201.9
214.7
231.6
226.8
177.2
149.5
97.0
86.2
76.2
102.8
117.1
88.9
84.2
114.9
134.4
123.4
164.3
14

In [61]:
y_pred

6653   NaN
6654   NaN
6655   NaN
6656   NaN
6657   NaN
6658   NaN
6659   NaN
6660   NaN
6661   NaN
6662   NaN
6663   NaN
6664   NaN
6665   NaN
6666   NaN
dtype: float64

In [59]:
train_list[j].ave

0        84.0
1       113.0
2        94.0
3       116.0
4        96.0
        ...  
6647    153.7
6649    173.0
6650    175.8
6651    138.4
6652    136.8
Name: ave, Length: 6647, dtype: float64

In [51]:
i = 0
for j in range(len(train_list)):
    for window in range(3,8):
        print('window:',window)
        regressor = rgs[i]
        name = names[i]
        forecaster = ReducedRegressionForecaster(
        regressor=regressor, window_length=window, strategy="recursive"
        )
        fh = ForecastingHorizon(test_list[j].index, is_relative=False)
        forecaster.fit(train_list[j].ave)
        y_pred = forecaster.predict(fh)
        mse = ((test_list[j].ave - y_pred) ** 2).mean()

        print(names[i], 'RMSE: {}'.format(round(np.sqrt(mse), 2)))
    print("\n")

window: 3
rfg RMSE: 91.44
window: 4
rfg RMSE: 78.17
window: 5
rfg RMSE: 55.61
window: 6
rfg RMSE: 62.89
window: 7
rfg RMSE: 64.95


window: 3
rfg RMSE: 35.89
window: 4
rfg RMSE: 41.3
window: 5
rfg RMSE: nan
window: 6
rfg RMSE: nan
window: 7


KeyboardInterrupt: 

In [ ]:

for i in range(len(rgs)):

    for window in range(3,15):
        print('window:',window)
        regressor = rgs[i]
        name = names[i]
        forecaster = ReducedRegressionForecaster(
        regressor=regressor, window_length=window, strategy="recursive"
        )
        fh = ForecastingHorizon(test.index, is_relative=False)
        forecaster.fit(train.ave)
        y_pred = forecaster.predict(fh)
        mse = ((test.ave - y_pred) ** 2).mean()

        print(names[i], 'RMSE: {}'.format(round(np.sqrt(mse), 2)))
    print('\n')

In [6]:
p_index = pd.RangeIndex(len(ts)+1,(len(ts)+1+14))
from sktime.forecasting.compose import EnsembleForecaster


In [8]:
arima_model = auto_arima(train.ave,
                      start_p=1, start_q=1,
                      max_p=5, max_q=5,
                      test='adf',        
                      trace=True,
                      alpha=0.05,
                      scoring='mse',
                      suppress_warnings=True,
                      seasonal = False
                      )



Performing stepwise search to minimize aic
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=60400.201, Time=1.13 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=85603.982, Time=0.12 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=60480.310, Time=0.26 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=77811.737, Time=0.68 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=60179.775, Time=4.43 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=inf, Time=0.34 sec
 ARIMA(3,0,1)(0,0,0)[0]             : AIC=inf, Time=5.60 sec
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=inf, Time=5.82 sec
 ARIMA(1,0,2)(0,0,0)[0]             : AIC=60320.550, Time=1.26 sec
 ARIMA(3,0,0)(0,0,0)[0]             : AIC=inf, Time=0.46 sec
 ARIMA(3,0,2)(0,0,0)[0]             : AIC=60163.235, Time=7.55 sec
 ARIMA(4,0,2)(0,0,0)[0]             : AIC=inf, Time=7.59 sec
 ARIMA(3,0,3)(0,0,0)[0]             : AIC=inf, Time=7.48 sec
 ARIMA(2,0,3)(0,0,0)[0]             : AIC=inf, Time=5.74 sec
 ARIMA(4,0,1)(0,0,0)[0]             : AIC=inf, Time=5.75 sec


In [9]:
arima_model = auto_arima(train.high,
                      start_p=1, start_q=1,
                      max_p=5, max_q=5,
                      test='adf',        
                      trace=True,
                      alpha=0.05,
                      scoring='mse',
                      suppress_warnings=True,
                      seasonal = False
                      )

Performing stepwise search to minimize aic
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=60577.379, Time=0.41 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=88417.836, Time=0.09 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=inf, Time=0.14 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=80391.769, Time=0.62 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=inf, Time=5.10 sec
 ARIMA(1,0,2)(0,0,0)[0]             : AIC=60498.897, Time=0.62 sec
 ARIMA(0,0,2)(0,0,0)[0]             : AIC=74831.837, Time=1.55 sec
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=inf, Time=6.34 sec
 ARIMA(1,0,3)(0,0,0)[0]             : AIC=60450.845, Time=1.48 sec
 ARIMA(0,0,3)(0,0,0)[0]             : AIC=71289.413, Time=2.78 sec
 ARIMA(2,0,3)(0,0,0)[0]             : AIC=inf, Time=5.87 sec
 ARIMA(1,0,4)(0,0,0)[0]             : AIC=60423.837, Time=0.88 sec
 ARIMA(0,0,4)(0,0,0)[0]             : AIC=68920.665, Time=5.10 sec
 ARIMA(2,0,4)(0,0,0)[0]             : AIC=inf, Time=10.35 sec
 ARIMA(1,0,5)(0,0,0)[0]             : AIC=60419.

In [11]:
arima_model = auto_arima(train.mid,
                      start_p=1, start_q=1,
                      max_p=5, max_q=5,
                      test='adf',        
                      trace=True,
                      alpha=0.05,
                      scoring='mse',
                      suppress_warnings=True,
                      seasonal = False
                      )

Performing stepwise search to minimize aic
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=61482.268, Time=0.39 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=85746.082, Time=0.10 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=61602.267, Time=0.33 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=78097.140, Time=0.92 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=61255.131, Time=2.76 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=61507.211, Time=0.27 sec
 ARIMA(3,0,1)(0,0,0)[0]             : AIC=inf, Time=9.22 sec
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=inf, Time=7.29 sec
 ARIMA(1,0,2)(0,0,0)[0]             : AIC=61396.830, Time=1.28 sec
 ARIMA(3,0,0)(0,0,0)[0]             : AIC=inf, Time=0.52 sec
 ARIMA(3,0,2)(0,0,0)[0]             : AIC=61238.337, Time=10.52 sec
 ARIMA(4,0,2)(0,0,0)[0]             : AIC=inf, Time=12.18 sec
 ARIMA(3,0,3)(0,0,0)[0]             : AIC=inf, Time=10.79 sec
 ARIMA(2,0,3)(0,0,0)[0]             : AIC=inf, Time=5.79 sec
 ARIMA(4,0,1)(0,0,0)[0]             : AIC=inf, Time=

In [10]:
arima_model = auto_arima(train.volume,
                      start_p=1, start_q=1,
                      max_p=5, max_q=5,
                      test='adf',        
                      trace=True,
                      alpha=0.05,
                      scoring='mse',
                      suppress_warnings=True,
                      seasonal = False
                      )

Performing stepwise search to minimize aic
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=96649.795, Time=1.62 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=109899.175, Time=0.14 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=97282.596, Time=0.29 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=104549.924, Time=0.93 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=96007.445, Time=4.76 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=96911.911, Time=0.39 sec
 ARIMA(3,0,1)(0,0,0)[0]             : AIC=96010.217, Time=3.80 sec
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=96003.947, Time=5.13 sec
 ARIMA(1,0,2)(0,0,0)[0]             : AIC=96234.376, Time=2.22 sec
 ARIMA(3,0,2)(0,0,0)[0]             : AIC=95986.448, Time=5.01 sec
 ARIMA(4,0,2)(0,0,0)[0]             : AIC=95988.742, Time=8.07 sec
 ARIMA(3,0,3)(0,0,0)[0]             : AIC=95988.927, Time=6.59 sec
 ARIMA(2,0,3)(0,0,0)[0]             : AIC=96006.873, Time=4.58 sec
 ARIMA(4,0,1)(0,0,0)[0]             : AIC=96007.598, Time=6.31 sec
 ARIMA(4,0,3)(0,0

In [7]:
mod = ARIMA(ts.ave, order=(2, 0, 1))
ave_results = mod.fit()
ave_fc, _, _ = ave_results.forecast(14, alpha=0.05)



mod = ARIMA(ts.ave, order=(2, 1, 1))
ave_results2 = mod.fit()
ave_fc2, _, _ = ave_results2.forecast(14, alpha=0.05)


ave_arima = np.mean(np.array([ave_fc,ave_fc2]),axis=0)
pd.Series(ave_arima).to_csv('FS443/ave_arima.csv', index = None, header=None)

In [66]:
mod = ARIMA(ts.high, order=(1, 0, 4))
high_results = mod.fit()
high_fc, _, _ = high_results.forecast(14, alpha=0.05)



mod = ARIMA(ts.high, order=(1, 1, 4))
high_results2 = mod.fit()
high_fc2, _, _ = high_results2.forecast(14, alpha=0.05)


high_arima = np.mean(np.array([high_fc,high_fc2]),axis=0)
pd.Series(high_arima).to_csv('FS443/high_arima.csv', index = None, header=None)

In [6]:
mod = ARIMA(ts.mid, order=(3, 0, 2))
mid_results = mod.fit(trend = 'nc')
mid_fc, _, _ = mid_results.forecast(14, alpha=0.05)



mod = ARIMA(ts.mid, order=(3, 1, 2))
mid_results2 = mod.fit(trend = 'nc')
mid_fc2, _, _ = mid_results2.forecast(14, alpha=0.05)


mid_arima = np.mean(np.array([mid_fc,mid_fc2]),axis=0)
pd.Series(mid_arima).to_csv('FS443/mid_arima.csv', index = None, header=None)

In [7]:
mod = ARIMA(ts.volume, order=(3, 0, 2))
volume_results = mod.fit(trend = 'nc')
volume_fc, _, _ = volume_results.forecast(14, alpha=0.05)



mod = ARIMA(ts.volume, order=(3, 1, 2))
volume_results2 = mod.fit(trend = 'nc')
volume_fc2, _, _ = volume_results2.forecast(14, alpha=0.05)


volume_arima = np.mean(np.array([volume_fc,volume_fc2]),axis=0)
pd.Series(volume_arima).to_csv('FS443/volume_arima.csv', index = None, header=None)

In [30]:

for i in range(len(rgs)):

    for window in range(3,15):
        print('window:',window)
        regressor = rgs[i]
        name = names[i]
        forecaster = ReducedRegressionForecaster(
        regressor=regressor, window_length=window, strategy="recursive"
        )
        fh = ForecastingHorizon(test.index, is_relative=False)
        forecaster.fit(train.ave)
        y_pred = forecaster.predict(fh)
        mse = ((test.ave - y_pred) ** 2).mean()

        print(names[i], 'RMSE: {}'.format(round(np.sqrt(mse), 2)))
    print('\n')

window: 3
rfg RMSE: 91.44
window: 4
rfg RMSE: 78.17
window: 5
rfg RMSE: 55.61
window: 6
rfg RMSE: 62.89
window: 7
rfg RMSE: 64.95
window: 8
rfg RMSE: 53.87
window: 9
rfg RMSE: 65.02
window: 10
rfg RMSE: 49.87
window: 11
rfg RMSE: 45.89
window: 12
rfg RMSE: 47.15
window: 13
rfg RMSE: 48.35
window: 14
rfg RMSE: 88.26


window: 3
Lasso RMSE: 71.41
window: 4
Lasso RMSE: 70.44
window: 5
Lasso RMSE: 70.29
window: 6
Lasso RMSE: 70.21
window: 7
Lasso RMSE: 70.37
window: 8
Lasso RMSE: 70.12
window: 9
Lasso RMSE: 70.26
window: 10
Lasso RMSE: 70.38
window: 11
Lasso RMSE: 71.08
window: 12
Lasso RMSE: 71.38
window: 13
Lasso RMSE: 70.8
window: 14
Lasso RMSE: 70.71


window: 3
XGB RMSE: 78.2
window: 4
XGB RMSE: 83.68
window: 5
XGB RMSE: 57.03
window: 6
XGB RMSE: 91.91
window: 7
XGB RMSE: 57.9
window: 8
XGB RMSE: 92.35
window: 9
XGB RMSE: 80.84
window: 10
XGB RMSE: 52.41
window: 11
XGB RMSE: 57.59
window: 12
XGB RMSE: 105.19
window: 13
XGB RMSE: 67.46
window: 14
XGB RMSE: 68.43


window: 3
KNN RMSE: 9

In [8]:
forecaster = EnsembleForecaster(
    [
        ("rfg", ReducedRegressionForecaster(regressor=rfg, window_length=11, strategy="recursive"),),
        ("lasso", ReducedRegressionForecaster(regressor=Lasso, window_length=8, strategy="recursive"),),
        ("xgb", ReducedRegressionForecaster(regressor=XGB, window_length=10, strategy="recursive"),),
        ("knn", ReducedRegressionForecaster(regressor=KNN, window_length=9, strategy="recursive"),),
        ("extra", ReducedRegressionForecaster(regressor=ExtraTree, window_length=13, strategy="recursive"),),
    ]
)
fh = ForecastingHorizon(p_index, is_relative=False)
forecaster.fit(ts.ave)
ave_ml = forecaster.predict(fh)

pd.Series(ave_ml).to_csv('FS443/ave_ml.csv', index = None, header=None)


In [31]:

for i in range(len(rgs)):

    for window in range(3,15):
        print('window:',window)
        regressor = rgs[i]
        name = names[i]
        forecaster = ReducedRegressionForecaster(
        regressor=regressor, window_length=window, strategy="recursive"
        )
        fh = ForecastingHorizon(test.index, is_relative=False)
        forecaster.fit(train.high)
        y_pred = forecaster.predict(fh)
        mse = ((test.high - y_pred) ** 2).mean()

        print(names[i], 'RMSE: {}'.format(round(np.sqrt(mse), 2)))
    print('\n')

window: 3
rfg RMSE: 80.46
window: 4
rfg RMSE: 78.28
window: 5
rfg RMSE: 72.72
window: 6
rfg RMSE: 69.98
window: 7
rfg RMSE: 84.22
window: 8
rfg RMSE: 83.96
window: 9
rfg RMSE: 62.86
window: 10
rfg RMSE: 72.89
window: 11
rfg RMSE: 65.09
window: 12
rfg RMSE: 80.16
window: 13
rfg RMSE: 69.51
window: 14
rfg RMSE: 67.18


window: 3
Lasso RMSE: 66.61
window: 4
Lasso RMSE: 66.44
window: 5
Lasso RMSE: 66.55
window: 6
Lasso RMSE: 66.6
window: 7
Lasso RMSE: 66.76
window: 8
Lasso RMSE: 66.34
window: 9
Lasso RMSE: 66.56
window: 10
Lasso RMSE: 67.3
window: 11
Lasso RMSE: 68.09
window: 12
Lasso RMSE: 67.89
window: 13
Lasso RMSE: 67.29
window: 14
Lasso RMSE: 67.25


window: 3
XGB RMSE: 73.89
window: 4
XGB RMSE: 83.38
window: 5
XGB RMSE: 56.71
window: 6
XGB RMSE: 72.43
window: 7
XGB RMSE: 83.33
window: 8
XGB RMSE: 105.79
window: 9
XGB RMSE: 91.98
window: 10
XGB RMSE: 80.13
window: 11
XGB RMSE: 95.79
window: 12
XGB RMSE: 75.95
window: 13
XGB RMSE: 85.27
window: 14
XGB RMSE: 52.87


window: 3
KNN RMSE: 

In [9]:
forecaster = EnsembleForecaster(
    [
        ("rfg", ReducedRegressionForecaster(regressor=rfg, window_length=9, strategy="recursive"),),
        ("lasso", ReducedRegressionForecaster(regressor=Lasso, window_length=8, strategy="recursive"),),
        ("xgb", ReducedRegressionForecaster(regressor=XGB, window_length=14, strategy="recursive"),),
        ("knn", ReducedRegressionForecaster(regressor=KNN, window_length=3, strategy="recursive"),),
        ("extra", ReducedRegressionForecaster(regressor=ExtraTree, window_length=9, strategy="recursive"),),
    ]
)
fh = ForecastingHorizon(p_index, is_relative=False)
forecaster.fit(ts.high)
high_ml = forecaster.predict(fh)

pd.Series(high_ml).to_csv('FS443/high_ml.csv', index = None, header=None)


In [32]:

for i in range(len(rgs)):

    for window in range(3,15):
        print('window:',window)
        regressor = rgs[i]
        name = names[i]
        forecaster = ReducedRegressionForecaster(
        regressor=regressor, window_length=window, strategy="recursive"
        )
        fh = ForecastingHorizon(test.index, is_relative=False)
        forecaster.fit(train.mid)
        y_pred = forecaster.predict(fh)
        mse = ((test.mid - y_pred) ** 2).mean()

        print(names[i], 'RMSE: {}'.format(round(np.sqrt(mse), 2)))
    print('\n')

window: 3
rfg RMSE: 77.53
window: 4
rfg RMSE: 78.45
window: 5
rfg RMSE: 79.73
window: 6
rfg RMSE: 77.99
window: 7
rfg RMSE: 80.58
window: 8
rfg RMSE: 80.2
window: 9
rfg RMSE: 86.43
window: 10
rfg RMSE: 83.27
window: 11
rfg RMSE: 88.16
window: 12
rfg RMSE: 84.39
window: 13
rfg RMSE: 87.39
window: 14
rfg RMSE: 81.03


window: 3
Lasso RMSE: 74.42
window: 4
Lasso RMSE: 73.06
window: 5
Lasso RMSE: 72.69
window: 6
Lasso RMSE: 72.55
window: 7
Lasso RMSE: 72.68
window: 8
Lasso RMSE: 72.35
window: 9
Lasso RMSE: 72.4
window: 10
Lasso RMSE: 72.55
window: 11
Lasso RMSE: 73.06
window: 12
Lasso RMSE: 73.54
window: 13
Lasso RMSE: 72.99
window: 14
Lasso RMSE: 72.93


window: 3
XGB RMSE: 93.43
window: 4
XGB RMSE: 108.34
window: 5
XGB RMSE: 109.34
window: 6
XGB RMSE: 110.05
window: 7
XGB RMSE: 85.2
window: 8
XGB RMSE: 64.1
window: 9
XGB RMSE: 84.41
window: 10
XGB RMSE: 74.0
window: 11
XGB RMSE: 74.21
window: 12
XGB RMSE: 101.51
window: 13
XGB RMSE: 91.71
window: 14
XGB RMSE: 72.02


window: 3
KNN RMSE: 

In [10]:
forecaster = EnsembleForecaster(
    [
        ("rfg", ReducedRegressionForecaster(regressor=rfg, window_length=6, strategy="recursive"),),
        ("lasso", ReducedRegressionForecaster(regressor=Lasso, window_length=8, strategy="recursive"),),
        ("xgb", ReducedRegressionForecaster(regressor=XGB, window_length=8, strategy="recursive"),),
        ("knn", ReducedRegressionForecaster(regressor=KNN, window_length=8, strategy="recursive"),),
        ("extra", ReducedRegressionForecaster(regressor=ExtraTree, window_length=11, strategy="recursive"),),
    ]
)
fh = ForecastingHorizon(p_index, is_relative=False)
forecaster.fit(ts.mid)
mid_ml = forecaster.predict(fh)

pd.Series(mid_ml).to_csv('FS443/mid_ml.csv', index = None, header=None)


In [33]:

for i in range(len(rgs)):

    for window in range(3,15):
        print('window:',window)
        regressor = rgs[i]
        name = names[i]
        forecaster = ReducedRegressionForecaster(
        regressor=regressor, window_length=window, strategy="recursive"
        )
        fh = ForecastingHorizon(test.index, is_relative=False)
        forecaster.fit(train.volume)
        y_pred = forecaster.predict(fh)
        mse = ((test.volume - y_pred) ** 2).mean()

        print(names[i], 'RMSE: {}'.format(round(np.sqrt(mse), 2)))
    print('\n')

window: 3
rfg RMSE: 28.09
window: 4
rfg RMSE: 261.52
window: 5
rfg RMSE: 232.37
window: 6
rfg RMSE: 73.4
window: 7
rfg RMSE: 154.43
window: 8
rfg RMSE: 71.56
window: 9
rfg RMSE: 57.37
window: 10
rfg RMSE: 56.84
window: 11
rfg RMSE: 29.52
window: 12
rfg RMSE: 35.72
window: 13
rfg RMSE: 33.1
window: 14
rfg RMSE: 32.66


window: 3
Lasso RMSE: 513.44
window: 4
Lasso RMSE: 448.91
window: 5
Lasso RMSE: 399.78
window: 6
Lasso RMSE: 345.78
window: 7
Lasso RMSE: 353.57
window: 8
Lasso RMSE: 323.81
window: 9
Lasso RMSE: 292.21
window: 10
Lasso RMSE: 263.07
window: 11
Lasso RMSE: 234.2
window: 12
Lasso RMSE: 205.03
window: 13
Lasso RMSE: 209.88
window: 14
Lasso RMSE: 208.71


window: 3
XGB RMSE: 373.16
window: 4
XGB RMSE: 228.78
window: 5
XGB RMSE: 180.69
window: 6
XGB RMSE: 98.46
window: 7
XGB RMSE: 156.8
window: 8
XGB RMSE: 59.9
window: 9
XGB RMSE: 209.54
window: 10
XGB RMSE: 177.15
window: 11
XGB RMSE: 78.85
window: 12
XGB RMSE: 58.53
window: 13
XGB RMSE: 48.7
window: 14
XGB RMSE: 52.79


wind

In [11]:
forecaster = EnsembleForecaster(
    [
        ("rfg", ReducedRegressionForecaster(regressor=rfg, window_length=3, strategy="recursive"),),
        ("lasso", ReducedRegressionForecaster(regressor=Lasso, window_length=12, strategy="recursive"),),
        ("xgb", ReducedRegressionForecaster(regressor=XGB, window_length=13, strategy="recursive"),),
        ("knn", ReducedRegressionForecaster(regressor=KNN, window_length=11, strategy="recursive"),),
        ("extra", ReducedRegressionForecaster(regressor=ExtraTree, window_length=10, strategy="recursive"),),
    ]
)
fh = ForecastingHorizon(p_index, is_relative=False)
forecaster.fit(ts.volume)
volume_ml = forecaster.predict(fh)

pd.Series(volume_ml).to_csv('FS443/volume_ml.csv', index = None, header=None)